In [8]:
def load_data():
    """
    Función utilitaria para cargar y preparar los datos del dataset.
    Esta función centraliza la carga de datos para evitar duplicación de código.
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍA PARA MANEJO DE DATOS
    # ============================================================================
    import pandas as pd  # Librería principal para manipulación de datos estructurados

    # ============================================================================
    # CARGA DEL DATASET DESDE ARCHIVO COMPRIMIDO
    # ============================================================================
    
    # Cargar el dataset de frases desde un archivo CSV comprimido
    # Utiliza ruta relativa desde la ubicación del notebook
    dataframe = pd.read_csv(
        "../files/input/sentences.csv.zip",  # Ruta relativa al archivo de datos comprimido
        index_col=False,  # No usar ninguna columna como índice del DataFrame
        compression="zip",  # Especificar que el archivo está en formato ZIP
    )

    # ============================================================================
    # SEPARACIÓN DE CARACTERÍSTICAS Y ETIQUETAS
    # ============================================================================
    
    # Extraer las frases de texto que serán las características (features) del modelo
    data = dataframe.phrase  # Columna con el texto a clasificar
    
    # Extraer las etiquetas objetivo que representan las clases verdaderas
    target = dataframe.target  # Columna con las categorías/clases de cada frase

    # ============================================================================
    # RETORNO DE DATOS SEPARADOS
    # ============================================================================
    
    # Devolver por separado las características y las etiquetas
    # Esto facilita su uso en otras funciones del pipeline
    return data, target


In [9]:
def make_train_test_split(x, y):
    """
    Función que divide los datos en conjuntos de entrenamiento y prueba.
    Utiliza una semilla aleatoria para garantizar reproducibilidad de resultados.
    
    Args:
        x: Características/features (frases de texto)
        y: Etiquetas/targets (categorías de clasificación)
    """

    # ============================================================================
    # IMPORTACIÓN DE FUNCIÓN PARA DIVISIÓN DE DATOS
    # ============================================================================
    from sklearn.model_selection import train_test_split  # Función para dividir datasets

    # ============================================================================
    # DIVISIÓN ESTRATIFICADA DE LOS DATOS
    # ============================================================================
    
    # Dividir el dataset en conjuntos de entrenamiento (75%) y prueba (25%)
    # La división es aleatoria pero reproducible gracias a random_state
    (x_train, x_test, y_train, y_test) = train_test_split(
        x,  # Datos de entrada (frases de texto)
        y,  # Etiquetas correspondientes
        test_size=0.25,  # 25% de los datos para conjunto de prueba
        random_state=123456,  # Semilla para reproducibilidad de la división aleatoria
    )
    
    # ============================================================================
    # RETORNO DE CONJUNTOS DIVIDIDOS
    # ============================================================================
    
    # Devolver los cuatro conjuntos resultantes:
    # - x_train, y_train: para entrenar el modelo
    # - x_test, y_test: para evaluar el rendimiento del modelo
    return x_train, x_test, y_train, y_test



In [10]:
def make_pipeline(estimator):
    """
    Función que construye un pipeline de procesamiento de texto completo.
    Crea una secuencia de transformaciones que preparan el texto para clasificación.
    
    Args:
        estimator: Algoritmo de machine learning a usar (ej: LogisticRegression)
    """

    # ============================================================================
    # IMPORTACIÓN DE COMPONENTES DEL PIPELINE
    # ============================================================================
    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer  # Para procesamiento de texto
    from sklearn.pipeline import Pipeline  # Para crear secuencias de transformaciones

    # ============================================================================
    # CONFIGURACIÓN DEL VECTORIZADOR DE TEXTO
    # ============================================================================
    
    # Primer componente: Convierte texto crudo en matriz numérica de conteos
    vectorizer = CountVectorizer(
        lowercase=True,  # Normalizar texto convirtiendo a minúsculas
        analyzer="word",  # Procesar a nivel de palabras individuales
        token_pattern=r"\b[a-zA-Z]\w+\b",  # Regex: palabras que empiecen con letra y contengan alfanuméricos
        stop_words="english",  # Filtrar palabras comunes del inglés (the, and, is, etc.)
    )

    # ============================================================================
    # CONFIGURACIÓN DEL TRANSFORMADOR TF-IDF
    # ============================================================================
    
    # Segundo componente: Convierte conteos brutos en pesos TF-IDF
    # TF-IDF da mayor peso a palabras discriminativas y menor a palabras comunes
    transformer = TfidfTransformer()

    # ============================================================================
    # CONSTRUCCIÓN DEL PIPELINE SECUENCIAL
    # ============================================================================
    
    # Crear pipeline que ejecuta transformaciones en orden secuencial:
    # texto → conteos → TF-IDF → clasificación
    pipeline = Pipeline(
        steps=[
            ("vectorizer", vectorizer),  # Paso 1: Convertir texto a matriz de conteos
            ("transformer", transformer),  # Paso 2: Aplicar transformación TF-IDF
            ("estimator", estimator),  # Paso 3: Aplicar algoritmo de clasificación
        ],
        verbose=False,  # No mostrar información detallada durante ejecución
    )
    
    # ============================================================================
    # RETORNO DEL PIPELINE CONFIGURADO
    # ============================================================================
    
    # Devolver el pipeline completo listo para entrenamiento
    return pipeline

In [11]:
def save_estimator(estimator):
    """
    Función que guarda un modelo entrenado en disco para uso posterior.
    Utiliza serialización binaria para preservar el estado completo del modelo.
    
    Args:
        estimator: Modelo/pipeline entrenado que se desea persistir
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍA PARA SERIALIZACIÓN
    # ============================================================================
    import pickle  # Librería estándar de Python para serialización de objetos

    # ============================================================================
    # PERSISTENCIA DEL MODELO EN DISCO
    # ============================================================================
    
    # Abrir archivo en modo escritura binaria para guardar el modelo
    # El contexto 'with' garantiza que el archivo se cierre correctamente
    with open("estimator.pickle", "wb") as file:
        # Serializar y escribir el modelo completo al archivo
        # Esto incluye el pipeline, parámetros entrenados y configuraciones
        pickle.dump(estimator, file)


In [12]:
def train_logistic_regression():
    """
    Función principal que orquesta todo el proceso de entrenamiento.
    Integra todas las funciones auxiliares para crear un flujo completo de ML.
    """

    # ============================================================================
    # IMPORTACIÓN DEL ALGORITMO DE CLASIFICACIÓN
    # ============================================================================
    from sklearn.linear_model import LogisticRegression  # Algoritmo de regresión logística

    # ============================================================================
    # PASO 1: CARGA DE DATOS
    # ============================================================================
    
    # Cargar el dataset completo usando la función utilitaria
    # Separa automáticamente características (frases) de etiquetas (categorías)
    data, target = load_data()

    # ============================================================================
    # PASO 2: DIVISIÓN DE DATOS EN ENTRENAMIENTO Y PRUEBA
    # ============================================================================
    
    # Dividir los datos en conjuntos de entrenamiento y prueba
    # Utiliza división aleatoria pero reproducible
    x_train, x_test, y_train, y_test = make_train_test_split(
        x=data,  # Frases de texto como características
        y=target,  # Categorías como etiquetas objetivo
    )

    # ============================================================================
    # PASO 3: CONSTRUCCIÓN DEL PIPELINE DE ML
    # ============================================================================
    
    # Crear pipeline completo que incluye:
    # - Procesamiento de texto (vectorización + TF-IDF)
    # - Algoritmo de clasificación (Regresión Logística)
    estimator = make_pipeline(estimator=LogisticRegression(max_iter=1000))
    # max_iter=1000: Número máximo de iteraciones para convergencia del algoritmo

    # ============================================================================
    # PASO 4: ENTRENAMIENTO DEL MODELO
    # ============================================================================
    
    # Entrenar el pipeline completo con los datos de entrenamiento
    # Esto ejecuta secuencialmente: vectorización → TF-IDF → entrenamiento del clasificador
    estimator.fit(x_train, y_train)

    # ============================================================================
    # PASO 5: PERSISTENCIA DEL MODELO ENTRENADO
    # ============================================================================
    
    # Guardar el modelo entrenado en disco para uso posterior
    # Esto evita tener que re-entrenar el modelo cada vez
    save_estimator(estimator)

# ============================================================================
# EJECUCIÓN DEL PROCESO COMPLETO DE ENTRENAMIENTO
# ============================================================================

# Ejecutar toda la secuencia de entrenamiento
# Este es el punto de entrada principal del proceso de ML
train_logistic_regression()


In [13]:
def use_estimator():
    """
    Función que utiliza un modelo previamente entrenado para generar predicciones.
    Implementa la fase de inferencia del proceso de machine learning.
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍAS NECESARIAS
    # ============================================================================
    import pickle  # Para cargar el modelo serializado desde disco
    import pandas as pd  # Para manejo de datos estructurados

    # ============================================================================
    # CARGA DE DATOS PARA REALIZAR PREDICCIONES
    # ============================================================================
    
    # Cargar el dataset de frases que queremos clasificar
    # En un escenario real, estos serían datos nuevos no vistos durante entrenamiento
    dataframe = pd.read_csv(
        "../files/input/sentences.csv.zip",  # Ruta relativa al archivo de datos
        index_col=False,  # No usar ninguna columna como índice
        compression="zip",  # El archivo está comprimido en formato ZIP
    )

    # ============================================================================
    # EXTRACCIÓN DE CARACTERÍSTICAS PARA PREDICCIÓN
    # ============================================================================
    
    # Extraer únicamente las frases de texto (no necesitamos las etiquetas)
    # Estas serán las características de entrada para el modelo
    data = dataframe.phrase

    # ============================================================================
    # CARGA DEL MODELO ENTRENADO DESDE DISCO
    # ============================================================================
    
    # Cargar el pipeline completo que fue guardado previamente
    # Esto incluye vectorizador, transformador TF-IDF y clasificador entrenados
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)  # Deserializar el modelo completo

    # ============================================================================
    # GENERACIÓN DE PREDICCIONES
    # ============================================================================
    
    # Aplicar el modelo a los datos para obtener predicciones de clase
    # El pipeline automáticamente ejecuta toda la secuencia:
    # 1. Vectorización del texto con el vocabulario aprendido
    # 2. Transformación TF-IDF con los parámetros del entrenamiento
    # 3. Clasificación usando el modelo entrenado
    prediction = estimator.predict(data)

    # ============================================================================
    # RETORNO DE PREDICCIONES
    # ============================================================================
    
    # Devolver las predicciones de clase para cada frase
    # Resultado: array con la clase predicha para cada entrada
    return prediction

# ============================================================================
# EJECUCIÓN DEL PROCESO DE INFERENCIA
# ============================================================================

# Ejecutar la función para generar predicciones usando el modelo entrenado
# Las predicciones se mostrarán como salida de la celda
use_estimator()


array(['neutral', 'positive', 'positive', ..., 'neutral', 'negative',
       'negative'], shape=(2264,), dtype=object)